In [ ]:
import fsspec
import dask.distributed

In [ ]:
import xarray as xr
import fsspec

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

?ebd.start_dask_cluster

In [ ]:
ebd.set_credentials(profile='esip-qhub')

In [ ]:
profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'
ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 10
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      environment='pangeo', worker_profile='Pangeo Worker', propagate_env=True)

In [ ]:
url = 's3://noaa-nwm-retro-v2-zarr-pds'

In [ ]:
fs1 = fsspec.filesystem('s3', anon=True)
fs2 = fsspec.filesystem('s3', anon=False)

In [ ]:
fs2.ls('s3://esip-qhub/noaa/NWM2')

In [ ]:
%%time
ds = xr.open_zarr(fs1.get_mapper(url), consolidated=True)

In [ ]:
cluster

In [ ]:
ds.streamflow

In [ ]:
ds2 = ds[['streamflow']].isel(time=slice(0,672),feature_id=slice(0,90000))

In [ ]:
ds2 = ds[['streamflow']].isel(time=slice(0,672))

In [ ]:
ds2.nbytes/1e6

In [ ]:
%%time
daily_mean = ds2.resample(time='D').mean().compute()

In [ ]:
%%time
ds2.coarsen(time=24).mean().compute()

In [ ]:
daily_mean.streamflow

In [ ]:
import numpy as np
np.reshape(ds2.streamflow,(24, 672//24, 2729077)).mean(dim=0)

In [ ]:
daily_mean.streamflow

In [ ]:
store_url = f's3://esip-qhub/noaa/NWM2/zarr/daily'
store = fs2.get_mapper(store_url)

In [ ]:
daily_mean = daily_mean.chunk({'time':31})

In [ ]:
daily_mean.streamflow

In [ ]:
def delete_s3(url):
    fs1 = fsspec.open(url, anon=False).fs
    if fs1.exists(url):
        fs1.rm(url, recursive=True)

In [ ]:
delete_s3(store_url)

In [ ]:
%%time
a = daily_mean.to_zarr(store, compute=False, mode='w', consolidated=True)

In [ ]:
%%time
dask.compute(a, retries=10)

In [ ]:
fs2.ls(f'{store_url}/streamflow')

In [ ]:
ds3 = xr.open_zarr(store)

In [ ]:
client.close(); cluster.shutdown()